In [ ]:
import pandas as pd
import random
import os
import sklearn
from sklearn.neighbors import NearestNeighbors
from math import radians

In [ ]:
static_df = pd.read_csv('data/datasets/static_variables.csv', index_col=[0])

In [ ]:
out_fire_df = None

n_s = [5]

for non_zero_n in n_s:
    out_fire_df = None
    for i in range(20):
        out_zero_df = None
        
        for root, dirs, files in os.walk('data/datasets/raw_datasets'):
            files = [file for file in files if file[:4] in ["2015", "2016", "2017", "2018", "2019"]]
            for file in files:
                print(file)
                out_df = pd.concat([static_df, pd.read_csv(os.path.join(root, file), index_col=[0])], axis=1)
                
                if i == 0:
                    fire_df = out_df.loc[out_df["TARGET"] == 1]
                    if out_fire_df is None:
                        if not fire_df.empty:
                            out_fire_df = fire_df
                    else:
                        out_fire_df = pd.concat([out_fire_df, fire_df])
                nonfire_df = out_df.loc[out_df["TARGET"] == 0]
                index = list(nonfire_df.index)
                sample = random.sample(index, non_zero_n)
                
                nearest_df = nonfire_df.copy()
                nearest_df["lat"] = nearest_df["lat"].apply(lambda x: radians(x))
                nearest_df["lon"] = nearest_df["lon"].apply(lambda x: radians(x))
                nbrs = NearestNeighbors(n_neighbors=1, algorithm="auto",
                                        metric='haversine').fit(nearest_df.loc[sample][["lon", "lat"]].values)
                distances, indices = nbrs.kneighbors(nearest_df[["lon", "lat"]].values)
                indices = indices.flatten()
                distances = distances * 6371
                
                incl_indices = set()
                
                for j, point in enumerate(sample):
                    nearest_sub_df = nearest_df.loc[indices == j].copy()
                    nearest_sub_df["d"] = distances[indices == j]
                    
                    incl_indices = incl_indices.union(set(nearest_sub_df["d"].nsmallest(100).index))
                
                incl_indices = list(incl_indices)
                
                if out_zero_df is None:
                    out_zero_df = out_df.loc[incl_indices]
                else:
                    out_zero_df = pd.concat([out_zero_df, out_df.loc[incl_indices]])
            
        plt.plot(*CA.exterior.xy)
        plt.scatter(out_zero_df["lon"].values, out_zero_df["lat"].values, s=0.1)
        plt.scatter(out_fire_df["lon"].values, out_fire_df["lat"].values, c='red', s=0.1, alpha=0.5)
        plt.show()
        pd.concat([out_zero_df, out_fire_df]).to_csv(f"data/datasets/final_datasets/{i}.csv")

### Validációs és teszt adathalmazok

In [ ]:
validation_set = None

for root, _, files in os.walk("data/datasets/raw_datasets"):
    files = [file for file in files if file.startswith("2020")]
    files = sorted(files, key=lambda x: int(x[:-4].split("-")[1]))
    for file in files:
        month_df = pd.concat([static_df, pd.read_csv(os.path.join(root, file), index_col=[0])], axis=1)
        if validation_set is None:
            validation_set = month_df
        else:
            validation_set = pd.concat([validation_set, month_df])
validation_set.index = [*range(5115924)]

validation_set.to_csv("data/datasets/validation_2020.csv")

del validation_set

test_set = None

for root, _, files in os.walk("data/datasets/raw_datasets"):
    files = [file for file in files if file.startswith("2021")]
    files = sorted(files, key=lambda x: int(x[:-4].split("-")[1]))
    for file in files:
        month_df = pd.concat([static_df, pd.read_csv(os.path.join(root, file), index_col=[0])], axis=1)
        if test_set is None:
            test_set = month_df
        else:
            test_set = pd.concat([test_set, month_df])
test_set.index = [*range(5115924)]

test_set.to_csv("data/datasets/test_2021.csv")